In [3]:
import pandas as pd
import os

file_path = os.path.expanduser("~/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/259/arxiv-metadata-oai-snapshot.json")

# 分块读取，加快速度、节省内存
chunks = pd.read_json(file_path, lines=True, chunksize=100000)

dfs = []
for chunk in chunks:
    dfs.append(chunk[["title", "categories"]])   # 只保留感兴趣的列

df_small = pd.concat(dfs, ignore_index=True)


                                               title       categories
0  Calculation of prompt diphoton production cros...           hep-ph
1           Sparsity-certifying Graph Decompositions    math.CO cs.CG
2  The evolution of the Earth-Moon system based o...   physics.gen-ph
3  A determinant of Stirling cycle numbers counts...          math.CO
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...  math.CA math.FA


In [4]:

print(df_small.head(20))

                                                title         categories
0   Calculation of prompt diphoton production cros...             hep-ph
1            Sparsity-certifying Graph Decompositions      math.CO cs.CG
2   The evolution of the Earth-Moon system based o...     physics.gen-ph
3   A determinant of Stirling cycle numbers counts...            math.CO
4   From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...    math.CA math.FA
5   Bosonic characters of atomic Cooper pairs acro...  cond-mat.mes-hall
6   Polymer Quantum Mechanics and its Continuum Limit              gr-qc
7   Numerical solution of shock and ramp compressi...  cond-mat.mtrl-sci
8   The Spitzer c2d Survey of Large, Nearby, Inste...           astro-ph
9   Partial cubes: structures, characterizations, ...            math.CO
10  Computing genus 2 Hilbert-Siegel modular forms...    math.NT math.AG
11  Distribution of integral Fourier Coefficients ...            math.NT
12  $p$-adic Limit of Weakly Holomorphic Modular F.

In [7]:
# Data Preprocessing
raw_df = df_small.rename(columns={"categories": "Category", "title": "Title"})
# Convert categories from string to list
raw_df["Category"] = raw_df["Category"].str.split(' ')
# Strip whitespace from titles
raw_df["Title"] = raw_df["Title"].str.strip()

print(raw_df.head(20))

                                                Title             Category
0   Calculation of prompt diphoton production cros...             [hep-ph]
1            Sparsity-certifying Graph Decompositions     [math.CO, cs.CG]
2   The evolution of the Earth-Moon system based o...     [physics.gen-ph]
3   A determinant of Stirling cycle numbers counts...            [math.CO]
4   From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   [math.CA, math.FA]
5   Bosonic characters of atomic Cooper pairs acro...  [cond-mat.mes-hall]
6   Polymer Quantum Mechanics and its Continuum Limit              [gr-qc]
7   Numerical solution of shock and ramp compressi...  [cond-mat.mtrl-sci]
8   The Spitzer c2d Survey of Large, Nearby, Inste...           [astro-ph]
9   Partial cubes: structures, characterizations, ...            [math.CO]
10  Computing genus 2 Hilbert-Siegel modular forms...   [math.NT, math.AG]
11  Distribution of integral Fourier Coefficients ...            [math.NT]
12  $p$-adic Limit of Wea

In [9]:
# Calculate the catrgory and its frequency
# Concactenate all categories into a single list
all_categories = [category for sublist in raw_df["Category"] for category in sublist]
# Calculate frequency
category_counts = pd.Series(all_categories).value_counts()
print(category_counts.head(20))
print(f"Total unique categories: {len(category_counts)}")

cs.LG                 240606
hep-ph                191472
hep-th                177619
cs.CV                 172242
quant-ph              169738
cs.AI                 149866
gr-qc                 117367
astro-ph              105380
cond-mat.mtrl-sci     103831
cond-mat.mes-hall      98127
cs.CL                  96563
math-ph                86649
math.MP                86649
cond-mat.str-el        80077
cond-mat.stat-mech     78891
math.CO                74424
astro-ph.CO            74148
stat.ML                74107
astro-ph.GA            73446
math.AP                70186
Name: count, dtype: int64
Total unique categories: 176


In [19]:
# 标签映射与数据集拆分（过滤单样本类别后）
from sklearn.model_selection import train_test_split

filtered_categories = category_counts[category_counts > 1].index.tolist()
filtered_df = raw_df[raw_df["Category"].apply(lambda cats: any(cat in filtered_categories for cat in cats))].reset_index(drop=True)
category_to_id = {cat: idx for idx, cat in enumerate(filtered_categories)}

# 为多标签分类创建 one-hot 编码标签
import numpy as np
def encode_labels(cats):
    labels = np.zeros(len(category_to_id), dtype=float)
    for cat in cats:
        if cat in category_to_id:
            labels[category_to_id[cat]] = 1.0
    return labels

filtered_df["label"] = filtered_df["Category"].apply(encode_labels)
train_df, test_df = train_test_split(filtered_df, test_size=0.05, random_state=42)
print(f"Training samples: {len(train_df)}, Testing samples: {len(test_df)}")
print(f"Number of labels: {len(category_to_id)}")


Training samples: 2734420, Testing samples: 143917
Number of labels: 176


In [27]:
# 转换为 HuggingFace Datasets 并分词
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# 减少训练样本数量以加快训练速度
# 原来：100,000 样本 -> 现在：10,000 样本 (10倍加速)
train_ds = Dataset.from_pandas(train_df.sample(n=10000, random_state=42).reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.sample(n=500, random_state=42).reset_index(drop=True))

# 减少序列长度以加快训练速度
# 原来：128 -> 现在：64 (约2倍加速)
max_length = 64

tokenizer_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

def tokenize(batch):
    tokenized = tokenizer(batch['Title'], truncation=True, padding='max_length', max_length=max_length)
    # 将 label 重命名为 labels（复数），这是 Hugging Face 模型期望的字段名
    tokenized['labels'] = batch['label']
    return tokenized

train_tokenized = train_ds.map(tokenize, batched=True)
test_tokenized = test_ds.map(tokenize, batched=True)

cols = ['input_ids','attention_mask','labels']
train_tokenized = train_tokenized.remove_columns([c for c in train_tokenized.column_names if c not in cols])
test_tokenized = test_tokenized.remove_columns([c for c in test_tokenized.column_names if c not in cols])

train_tokenized.set_format(type='torch')
test_tokenized.set_format(type='torch')

print(f'训练集: {len(train_tokenized)} 样本, 测试集: {len(test_tokenized)} 样本')
print(f'序列最大长度: {max_length}')
print(train_tokenized, test_tokenized)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

训练集: 10000 样本, 测试集: 500 样本
序列最大长度: 64
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


In [24]:
# 初始化模型（使用基础 distilbert 而非 SST-2 头，配置为多标签分类）
from transformers import AutoConfig, AutoModelForSequenceClassification

base_model_name = 'distilbert/distilbert-base-uncased'

config = AutoConfig.from_pretrained(base_model_name)
# 设置标签数量和问题类型为多标签分类
config.num_labels = len(category_to_id)
config.problem_type = "multi_label_classification"

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
    ignore_mismatched_sizes=True  # 允许重新初始化分类头
)

print('模型已初始化，标签数:', model.config.num_labels)
print('问题类型:', model.config.problem_type)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


模型已初始化，标签数: 176
问题类型: multi_label_classification


In [28]:
# 训练设置与 Trainer 初始化 (只需运行一次)
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score

output_dir = 'model/checkpoint'

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = sigmoid(logits)
    preds = (probs > 0.5).astype(int)
    
    # 计算多标签指标
    # 样本级准确率：预测完全匹配的比例
    exact_match = (preds == labels).all(axis=1).mean()
    
    # 微平均和宏平均 F1
    f1_micro = f1_score(labels, preds, average='micro', zero_division=0)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0)
    
    return {
        'exact_match': exact_match,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro
    }

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=256,  # 增大 batch size (128 -> 256) 加速训练
    per_device_eval_batch_size=256,
    num_train_epochs=1,  # 每次只跑 1 个 epoch,可以手动多次运行
    learning_rate=5e-5,
    eval_strategy='epoch',  # 每个 epoch 结束后评估
    save_strategy='epoch',
    logging_steps=5,  # 减少日志频率 (10 -> 5)
    load_best_model_at_end=True,
    metric_for_best_model='f1_micro',
    save_total_limit=2,
    report_to='none',
    use_mps_device=True,  # 使用 Apple Silicon MPS
    fp16=False,  # MPS 不支持 fp16，但可以考虑使用 bf16
    gradient_accumulation_steps=1,  # 保持默认
    dataloader_num_workers=0,  # MPS 使用 0 比较稳定
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

print('Trainer 已初始化,准备开始训练')
print(f'训练配置: batch_size={training_args.per_device_train_batch_size}, max_length={max_length}')
print(f'预计训练步数: {len(train_tokenized) // training_args.per_device_train_batch_size} 步/epoch')


Trainer 已初始化,准备开始训练
训练配置: batch_size=256, max_length=64
预计训练步数: 39 步/epoch


/Users/jylu/Projects/acafeed/.venv/lib/python3.14/site-packages/transformers/training_args.py:2301: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [43]:
# 执行训练 (可重复运行此单元格进行多次 epoch 训练)
trainer.train()


/Users/jylu/Projects/acafeed/.venv/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Exact Match,F1 Micro,F1 Macro
1,0.051300,0.051604,0.000000,0.000000,0.000000


TrainOutput(global_step=40, training_loss=0.054133594036102295, metrics={'train_runtime': 79.5518, 'train_samples_per_second': 125.704, 'train_steps_per_second': 0.503, 'total_flos': 166098063360000.0, 'train_loss': 0.054133594036102295, 'epoch': 1.0})

In [33]:
# 评估与分类报告
from sklearn.metrics import classification_report

metrics = trainer.evaluate()
print('Eval metrics:', metrics)

/Users/jylu/Projects/acafeed/.venv/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Eval metrics: {'eval_loss': 0.09000758081674576, 'eval_exact_match': 0.0, 'eval_f1_micro': 0.0, 'eval_f1_macro': 0.0, 'eval_runtime': 1.7333, 'eval_samples_per_second': 288.467, 'eval_steps_per_second': 1.154, 'epoch': 1.0}


In [44]:
import torch

# 创建 id 到标签的映射
id2label = {v: k for k, v in category_to_id.items()}

# 推理示例
sample_titles = [
    raw_df.iloc[0]['Title'],
    raw_df.iloc[10]['Title'],
    raw_df.iloc[100]['Title']
]

# 获取模型所在的设备
device = model.device
print(f'模型设备: {device}')

model.eval()
# DistilBERT 不支持 token_type_ids，需要移除
encoded = tokenizer(sample_titles, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
# 移除 token_type_ids（如果存在）
if 'token_type_ids' in encoded:
    encoded.pop('token_type_ids')

# 将输入张量移动到模型所在的设备（MPS）
encoded = {k: v.to(device) for k, v in encoded.items()}

with torch.no_grad():
    outputs = model(**encoded)
    # 多标签分类使用 sigmoid 而非 softmax
    probs = torch.sigmoid(outputs.logits)
    # 将结果移回 CPU 进行后处理
    probs = probs.cpu()

for title, prob in zip(sample_titles, probs):
    # 获取概率 > 0.5 的所有标签
    pred_indices = (prob > 0.5).nonzero(as_tuple=True)[0].tolist()
    pred_labels = [(id2label[i], float(prob[i])) for i in pred_indices]
    
    # 获取 Top 5 最高概率的标签
    top5 = sorted([(id2label[i], float(p)) for i, p in enumerate(prob)], key=lambda x: x[1], reverse=True)[:5]
    
    print('\n' + '='*80)
    print('Title:', title[:100])
    print(f'预测标签 (>0.5): {len(pred_labels)} 个')
    for label, score in pred_labels:
        print(f'  - {label}: {score:.3f}')
    print('\nTop 5 标签:')
    for label, score in top5:
        print(f'  - {label}: {score:.3f}')

模型设备: mps:0

Title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
预测标签 (>0.5): 0 个

Top 5 标签:
  - cs.LG: 0.073
  - hep-ph: 0.056
  - hep-th: 0.052
  - quant-ph: 0.050
  - cs.CV: 0.049

Title: Computing genus 2 Hilbert-Siegel modular forms over $\Q(\sqrt{5})$ via
  the Jacquet-Langlands corre
预测标签 (>0.5): 0 个

Top 5 标签:
  - cs.LG: 0.073
  - hep-ph: 0.055
  - hep-th: 0.052
  - quant-ph: 0.051
  - cs.CV: 0.049

Title: The birth of string theory
预测标签 (>0.5): 0 个

Top 5 标签:
  - cs.LG: 0.074
  - hep-ph: 0.056
  - hep-th: 0.053
  - quant-ph: 0.051
  - cs.CV: 0.049
